In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import interpolate
from scipy import integrate
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from math import sqrt
from math import pi
import matplotlib.font_manager as fm
import matplotlib.ticker as mtick
from matplotlib.ticker import ScalarFormatter
from scipy import optimize
from labellines import labelLine, labelLines
import csv

### This notebook contains the RHS functions for eqs 3-6 of the "MarshakWave 3T" notebook and the corresponding initial conditions.

In [4]:
#coupling coefficient  function
gamma_val = lambda x, xmax: gamma(x,g(x,xmax))
gamma0=0
gamma = lambda t,Te: gamma0*((Te)**(-m))
#initial condition g for the electron temperature
g = lambda x, xmax: ((n+3)*xmax*(-x+xmax)/(n+4))**(1/(n+3))
gprime = lambda x, xmax: -(((n+3)/(n+4))*xmax)**(1/(n+3))*1/(n+3)*(xmax-x)**((-2-n)/(n+3))

# IC for time dependent Ti
#this function returns the IC for Te if the two temperatures are fully coupled
h_hi = g
h_low = lambda x, xmax: 1/(Cvi)*gamma_val(x,xmax)*((n+3)/(n+4))*((xmax-x)/(xmax))*((n+3)/(n+4)*xmax*(xmax-x))**(1/(n+3))
h=lambda x, xmax: min((h_hi(x,xmax),h_low(x,xmax)))

#IC for Ti eq 6 
f=lambda x, xmax:4*gamma_val(x,xmax)*(n+3)*(-xmax+x)*((n+3)*xmax*(-x+xmax)/(n+4))**(1/(n+3))/(Cvi*(n+4)*(2*gamma_val(x,xmax)*xmax**2/Cvi-x**3-4*gamma_val(x,xmax)*x*xmax/Cvi-x**2*xmax+2*gamma_val(x,xmax)*xmax**2/Cvi-x*xmax-xmax**3))

### RHS functions 
To make eqs 3-6 first order in terms of the x derivative, the variable u is defined,
$$ u = \frac{dT_e}{d\xi}$$
The following right hand side functions are eqs 3-6 rearranged to solve for $\frac{du}{d\xi}$.

In [5]:
#Vector functions for solving eq's 3,4 and 5,6
#v[0] = Te, v[1] = u, v[2] = Ti
def RHS_time(t,v,gamma):
    Te = v[0]
    Ti = v[2]
    gamma_val = gamma(t,Te)
    result = np.zeros(3)
    #compute RHS
    result[0] = v[1]
    result[1] = ((-t*v[1] - 1/(Cve)*gamma_val*(v[2]-v[0]))*(v[0]**(-n))-(n+4)*(n+3)*(v[1]**2)*(v[0]**2))/((n+4)*v[0]**3) #eq 3
    result[2] = gamma_val/(Cvi*t)*(v[2]-v[0])  #eq 4
    return result

# Space dependent gamma
def RHS_space(t,v,gamma):
    Te = v[0]
    Ti = v[2]
    gamma_val = gamma(t,Te)
    result = np.zeros(3)
    #compute RHS
    result[0] = v[1]
    result[1] = (-t*v[1]*v[0]**(-n)-gamma_val/(Cve*t**2)*v[0]**(-n)*(v[2]-v[0])-(n+3)*(n+4)*v[1]**2*v[0]**2)/((n+4)*v[0]**3)  #eq 5
    result[2] = (gamma_val)/(Cvi*t**3)*(v[2]-v[0])  #eq 6
    return result


In [6]:
RHSfun_time = lambda t,v: RHS_time(t,v,gamma)
RHSfun_space = lambda t,v: RHS_space(t,v,gamma)
